In [31]:
# -*- coding: utf-8 -*-
import nltk
import string
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import pandas

path = '../crawler/ksou/data/description/'
token_dict = {}
stemmer = PorterStemmer()
translator = str.maketrans('', '', string.punctuation)
count = 0
LIMIT = 1000
re_number = re.compile(r'\d')

def has_number(string):
    return re_number.search(string) != None

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    stems = [s for s in stems if not has_number(s)]
#    print(stems)
    return stems

def getResult():
    global count
    for file in os.listdir(path):
        with open(path+file, 'rb') as f:
            text = f.read().decode('utf-8')
    #        print(text)
            if len(text) > 20:
                token_dict[file] = text.translate(translator)
                count += 1
                if count >= LIMIT:
                    break
    
    #this can take some time
    print('Finish collecting the doc... training using tfidf')
    tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', lowercase=True)
    tfs = tfidf.fit_transform(token_dict.values())
    dense = tfs.todense()
    feature_names = tfidf.get_feature_names()
    doc_names = token_dict.keys()
    return dense, feature_names, doc_names



In [32]:
dense, feature_names, doc_names = getResult()

Finish collecting the doc... training using tfidf


In [52]:
df = pandas.DataFrame(dense, index = doc_names, columns = feature_names)
df_non_zero = df[df !=0]
df_non_zero_sum = (df !=0).sum(axis = 0)
# df.to_csv('./result/'+'100_tfid_result.csv')
df_non_zero_100 = df_non_zero_sum[df_non_zero_sum > 100].sort_values(ascending=False)
df_non_zero_100

thi          828
bedroom      817
home         697
kitchen      645
bathroom     615
area         585
live         520
locat        512
featur       506
block        458
room         447
open         438
face         389
floor        381
larg         380
secur        360
land         359
frontag      358
metr         356
park         356
backyard     354
loung        345
level        345
view         343
ha           342
plan         341
properti     339
famili       336
air          336
slope        334
            ... 
south        133
bath         129
ideal        128
contact      127
stainless    127
style        126
stun         123
steel        123
meal         122
ground       121
aircondit    119
window       118
today        117
fantast      115
distanc      115
entri        115
dishwash     114
carport      113
complet      112
buyer        111
year         110
price        110
pleas        110
short        108
bay          107
inform       107
care         105
shed         1

In [71]:
non_zero_100_cols = df_non_zero[df_non_zero_100.index]
non_zero_100_cols_average = non_zero_100_cols.sum(axis = 0)/df_non_zero_sum[df_non_zero_100.index]
non_zero_100_cols.to_csv('./result/100_words.csv')

PermissionError: [Errno 13] Permission denied: './result/100_words.csv'